In [1]:
import neuroglancer
import numpy as np

Create a new (initially empty) viewer.  This starts a webserver in a background thread, which serves a copy of the Neuroglancer client, and which also can serve local volume data and handles sending and receiving Neuroglancer state updates.

In [2]:
viewer = neuroglancer.Viewer()

Print a link to the viewer (only valid while the notebook kernel is running). Note that while the Viewer is running, anyone with the link can obtain any authentication credentials that the neuroglancer Python module obtains. Therefore, be very careful about sharing the link, and keep in mind that sharing the notebook will likely also share viewer links.

In [4]:
viewer

http://127.0.0.1:37179/v/5b2b18aaa30a266d7fbcec20bd6f321cc089f3e1/

Add some example layers using the precomputed data source (HHMI Janelia FlyEM FIB-25 dataset).

In [5]:
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em')
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://https://storage.googleapis.com/iarpa_microns/minnie/minnie65/seg_m943/', selected_alpha=0.3) # graphene://middleauth+https://minnie.microns-daf.com/segmentation/table/minnie65_public 
  # skeleton layer only works in the Allen Institute network
  s.layers['skeleton'] = neuroglancer.SegmentationLayer(source='precomputed://http://bigkahuna.corp.alleninstitute.org/ACdata/Users/wanqing/Neuroglancer/skel_axon/skeletons/') 
  s.dimensions = neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[4, 4, 40])


In [6]:
# 864691135122603047	292685	L2a
# 864691135639556411_267207_L5b
rid = 864691136903144370 #864691135162983725 #864691135639556411 # 864691136134678667 #864691135122603047 #864691135755817170	 # 864691134884807418 #864691135162983725 864691135755817170 	
sid = 236197 #267207 # 271518 #292685 #588983 #518848 #495010 588983
cell_type = 'L6short-a' #'L5ET' #'L5b' # 'L6tall-a' #'L2a' 		
soma_pos = [152681, 226226, 19189]

In [7]:
# make a cetain segment visible
with viewer.txn() as s:
  s.layers['segmentation'].segments = [rid]
  s.layers['skeleton'].segments = [rid]
  s.voxel_coordinates = soma_pos

In [8]:
# set up path for skeletons

import skeleton_plot as skelplot
import skeleton_plot.skel_io as skel_io

#raw skeleton files
skel_path = 'https://storage.googleapis.com/allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all/v661/skeletons/'

skel_dir = str(rid)+'_'+str(sid) #'864691134884807418_518848'
skel_filename = str(rid)+'_'+str(sid)+'.swc' #'864691134884807418_518848.swc'

# raw skel
skel = skel_io.read_skeleton(skel_path, skel_filename)
# get the axon segment out
axon_indices = np.where(skel.vertex_properties['compartment'] == 2)[0]
# skel.vertices


/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [9]:
import navis
skel_navis = navis.read_swc(skel_path+skel_filename)
skel_navis

,
type,navis.TreeNeuron
name,864691136903144370_236197
n_nodes,7805
n_connectors,None
n_branches,39
n_leafs,51
cable_length,11642.525391
soma,[0]
units,1 dimensionless
created_at,2024-08-16 11:39:35.728983


In [10]:
len(skel_navis.segments[40])

61

In [11]:
skel_seg1 = navis.TreeNeuron(skel_navis.nodes[skel_navis.nodes.node_id.isin(skel_navis.segments[0])])
skel_seg1



/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/navis/graph/graph_utils.py:399: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.nodes['type'] = 'slab'
/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/navis/graph/graph_utils.py:412: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.nodes['type'] = x.nodes['type'].astype(cat_types)


,
type,navis.TreeNeuron
name,None
n_nodes,708
n_connectors,None
n_branches,0
n_leafs,1
cable_length,1054.874756
soma,[0]
units,1 dimensionless


In [12]:
skel_seg1.nodes

,node_id,label,x,y,z,radius,parent_id,type
0,0,1,609.664001,904.512024,772.400024,5.805,-1,root
1,1,2,609.341980,911.786011,774.395020,0.224,0,slab
2,2,2,609.278015,913.241028,774.794983,0.224,1,slab
3,3,2,609.213013,914.695984,775.193970,0.224,2,slab
4,4,2,609.148987,916.151001,775.593018,0.224,3,slab
...,...,...,...,...,...,...,...,...
703,703,2,1456.977051,774.052979,769.138977,0.139,702,slab
704,704,2,1458.479980,773.953979,769.028992,0.139,703,slab
705,705,2,1459.972046,773.958008,768.812012,0.139,704,slab
706,706,2,1461.462036,774.013977,768.599976,0.139,705,slab


In [13]:
skel_seg1.nodes['x'], skel_seg1.nodes['y'], skel_seg1.nodes['z'] = skel_seg1.nodes['x'] * 1000 / 4, skel_seg1.nodes['y'] * 1000 / 4, skel_seg1.nodes['z'] * 1000 / 40
skel_seg1.nodes

/tmp/ipykernel_4040241/1175870376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skel_seg1.nodes['x'], skel_seg1.nodes['y'], skel_seg1.nodes['z'] = skel_seg1.nodes['x'] * 1000 / 4, skel_seg1.nodes['y'] * 1000 / 4, skel_seg1.nodes['z'] * 1000 / 40


,node_id,label,x,y,z,radius,parent_id,type
0,0,1,152416.00,226128.00,19310.000000,5.805,-1,root
1,1,2,152335.50,227946.50,19359.875000,0.224,0,slab
2,2,2,152319.50,228310.25,19369.875000,0.224,1,slab
3,3,2,152303.25,228674.00,19379.849609,0.224,2,slab
4,4,2,152287.25,229037.75,19389.824219,0.224,3,slab
...,...,...,...,...,...,...,...,...
703,703,2,364244.25,193513.25,19228.474609,0.139,702,slab
704,704,2,364620.00,193488.50,19225.724609,0.139,703,slab
705,705,2,364993.00,193489.50,19220.300781,0.139,704,slab
706,706,2,365365.50,193503.50,19215.000000,0.139,705,slab


In [14]:
import copy
import time

def interpolate_to(final_state, frames_per_second=5, seconds=1):
    total_frames = int(round(seconds * frames_per_second))
    initial_state = viewer.state
    for frame_i in range(total_frames):
        t = frame_i / total_frames
        viewer.set_state(
            neuroglancer.ViewerState.interpolate(initial_state, final_state, t)
        )
        time.sleep(1 / frames_per_second)
    viewer.set_state(final_state)

def move_by(offset, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates += offset
    interpolate_to(final_state, **kwargs)

def move_to(final_voxel_coordinates, final_orientation, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates = final_voxel_coordinates
    final_state.crossSectionOrientation = final_orientation
    final_state.projectionOrientation = final_orientation
    interpolate_to(final_state, **kwargs)

def do_move_by():
    move_by([100, 100, 100], seconds=1, frames_per_second=10)

def do_move_to(final_voxel_coordinates, final_orientation, seconds=1, frames_per_second=10):
    move_to(final_voxel_coordinates, final_orientation = final_orientation, seconds=seconds, frames_per_second=frames_per_second)


In [15]:
len(skel_seg1.nodes)

708

In [18]:
# move along the axon
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_seg1.nodes['x'][0], skel_seg1.nodes['y'][0], skel_seg1.nodes['z'][0]]

In [19]:
# do move to according to the list of positions and orientations in df
for i in range(40): #len(skel_seg1.nodes)
    do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation = [0,0,1,0], seconds = 2)

KeyboardInterrupt: 

In [85]:
# smooth the skeleton
skel_seg1_sm = navis.smooth_skeleton(skel_seg1, window=10, to_smooth=['x', 'y', 'z'])
fig = skel_seg1_sm.plot3d(backend='plotly', connectors=False)

Smoothing:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
# get the dotprops
skel_dp = navis.make_dotprops(skel_seg1, k=5)
skel_dp

,
type,navis.Dotprops
name,None
k,5
units,1 dimensionless
n_points,708


In [21]:
import numpy as np

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

def surface_normal_to_quaternion(normal):
    normal = normalize(normal)
    up_vector = np.array([0, 0, 1])
    axis = np.cross(up_vector, normal)
    axis = normalize(axis)
    angle = np.arccos(np.dot(up_vector, normal))
    
    qx = axis[0] * np.sin(angle / 2)
    qy = axis[1] * np.sin(angle / 2)
    qz = axis[2] * np.sin(angle / 2)
    qw = np.cos(angle / 2)
    
    return [-qx, -qy, -qz, qw]


In [22]:
skel_dp.vect[0]

array([-0.04419616,  0.99864715,  0.02739745], dtype=float32)

In [23]:
import math

def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    cos_angle = np.clip(cos_angle, -1, 1)  # Ensure within valid range for arccos
    angle_radians = np.arccos(cos_angle)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

def flip_vectors_if_necessary(skel_dp_vect):
    """Flip the direction of vectors if the angle with the previous vector is > 150 degrees."""
    for i in range(1, len(skel_dp_vect)):
        angle = angle_between_vectors(skel_dp_vect[i], skel_dp_vect[i-1])
        if angle > 150:
            skel_dp_vect[i] = abs(skel_dp_vect[i]) * np.sign(skel_dp_vect[i-1])
    return skel_dp_vect


def quaternion_to_rotation_angle(q1, q2):
    # Normalize the quaternions
    q1 = q1 / np.linalg.norm(q1)
    q2 = q2 / np.linalg.norm(q2)
    
    # Compute the dot product
    dot_product = np.dot(q1, q2)
    
    # Ensure the dot product is within the valid range for arccos
    dot_product = np.clip(dot_product, -1.0, 1.0)
    
    # Calculate the angle in radians
    angle_rad = 2 * np.arccos(dot_product)
    
    # Convert the angle to degrees
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

def flip_quaternion_axis(quaternion):
    """Flip the axis of a quaternion by negating the x, y, and z components."""
    for i in range(1, len(quaternion)):
        angle = quaternion_to_rotation_angle(quaternion[i], quaternion[i-1])
        if angle > 130:
            quaternion[i] = np.array([-quaternion[i,0], -quaternion[i,1], -quaternion[i,2], quaternion[i,3]])
    return quaternion

In [24]:
skel_dp.vect = flip_vectors_if_necessary(skel_dp.vect)

In [25]:
# get the quaternion for each node
quaternions = []
for i in range(len(skel_dp)):
    quaternions.append(surface_normal_to_quaternion(skel_dp.vect[i]))
quaternions

[[0.6966711340272986, 0.030831898794778927, -0.0, 0.7167277900499088],
 [0.6966162078763772, 0.01781052035055687, -0.0, 0.7172228693291774],
 [0.6966162078763772, 0.01781052035055687, -0.0, 0.7172228693291774],
 [0.6966162064407941, 0.0178105242093893, -0.0, 0.7172228706276895],
 [0.6908174857738053, 0.06351442873829802, -0.0, 0.7202340721537728],
 [0.6478451047500658, 0.2373615457053603, -0.0, 0.7238482001578359],
 [-0.5775731790607984, 0.43975186943680844, -0.0, 0.6877699587481523],
 [-0.4992716213149154, 0.5261497835943959, -0.0, 0.6883997772901631],
 [-0.5142262385288706, 0.5107133234741004, -0.0, 0.689016165873115],
 [-0.5729777144900488, 0.44556612463048695, -0.0, 0.6878716212197807],
 [-0.6257213388150559, 0.3686196404048177, -0.0, 0.6874535379640699],
 [-0.6586852751278324, 0.2996604003904838, -0.0, 0.6901719733280879],
 [-0.67457250372801, 0.24722298337428908, -0.0, 0.6955808606521889],
 [-0.692590483936193, 0.17460357569909263, -0.0, 0.699879998938476],
 [-0.7065968895080269,

In [26]:
# flip the quaternion if necessary
fixed_quaternions = flip_quaternion_axis(np.array(quaternions))
fixed_quaternions

array([[ 0.69667113,  0.0308319 , -0.        ,  0.71672779],
       [ 0.69661621,  0.01781052, -0.        ,  0.71722287],
       [ 0.69661621,  0.01781052, -0.        ,  0.71722287],
       ...,
       [-0.0088587 ,  0.70330423, -0.        ,  0.7108338 ],
       [-0.0088587 ,  0.70330423, -0.        ,  0.7108338 ],
       [-0.0088587 ,  0.70330423, -0.        ,  0.7108338 ]])

In [27]:
# move along the axon
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_seg1.nodes['x'][0], skel_seg1.nodes['y'][0], skel_seg1.nodes['z'][0]]
    s.crossSectionOrientation = fixed_quaternions[0]

In [28]:
# do move to according to the list of positions and orientations in df
for i in range(len(skel_seg1.nodes)): #len(skel_seg1.nodes)
    do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation = fixed_quaternions[i], seconds = 2)

KeyboardInterrupt: 

In [38]:
from ipywidgets import Button, VBox, Output
from IPython.display import display
import time

# Initialize widgets
btn_pause = Button(description='Pause')
btn_continue = Button(description='Continue', disabled=True)
btn_back = Button(description='Back', disabled=True)
btn_forth = Button(description='Forth', disabled=True)
output = Output()

# Global control variables
# i = 0
paused = False

# def do_move_to(position, final_orientation, seconds):
#     # Placeholder for the actual movement function
#     with output:
#         print(f"Moving to {position} with orientation {final_orientation} in {seconds} seconds")

def update_buttons():
    global i
    btn_back.disabled = i <= 0
    btn_forth.disabled = i >= len(skel_seg1.nodes) - 1
    btn_continue.disabled = not paused

def on_pause_clicked(b):
    global paused
    paused = True
    update_buttons()

def on_continue_clicked(b):
    global paused
    paused = False
    update_buttons()

def on_back_clicked(b):
    global i
    if i > 0:
        i -= 1
        do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

def on_forth_clicked(b):
    global i
    if i < len(skel_seg1.nodes) - 1:
        i += 1
        do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

# Attach event handlers
btn_pause.on_click(on_pause_clicked)
btn_continue.on_click(on_continue_clicked)
btn_back.on_click(on_back_clicked)
btn_forth.on_click(on_forth_clicked)

# Display widgets
display(VBox([btn_pause, btn_continue, btn_back, btn_forth, output]))

# Initial execution
do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)

Create a publicly sharable URL to the viewer state (only works for external data sources, not layers served from Python).  The Python objects for representing the viewer state (`neuroglancer.ViewerState` and friends) can also be used independently from the interactive Python-tied viewer to create Neuroglancer links.

Stop the Neuroglancer web server, which invalidates any existing links to the Python-tied viewer.

In [30]:
neuroglancer.stop()